In [19]:
from transformers import TFAutoModel , AutoTokenizer, AdamW

In [20]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('/content/Untitled 1.csv')

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['Priority'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['Priority'])

In [26]:
label_map = {'Low': 0, 'Medium': 1, 'High': 2}
train_df['label'] = train_df['Priority'].map(label_map)
val_df['label'] = val_df['Priority'].map(label_map)
test_df['label'] = test_df['Priority'].map(label_map)

In [27]:
from datasets import Dataset
import tensorflow as tf

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
from transformers import BertTokenizer
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True, max_length=128)
train_dataset = Dataset.from_pandas(train_df[['Text', 'label']])
val_dataset = Dataset.from_pandas(val_df[['Text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Text', 'label']])
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_tf_dataset = train_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    batch_size=16
)
val_tf_dataset = val_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    batch_size=16
)
test_tf_dataset = test_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    batch_size=16
)

In [ ]:
# Train the model
model.fit(train_tf_dataset, epochs=50, validation_data=val_tf_dataset)


In [ ]:
test_loss, test_accuracy = model.evaluate(test_tf_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
model.save("model")